# LLM Evaluation with MLflow Example Notebook

In this notebook, we will demonstrate how to evaluate various LLMs and RAG systems with MLflow, leveraging simple metrics such as toxicity, as well as LLM-judged metrics such as relevance, and even custom LLM-judged metrics such as professionalism

We need to set our OpenAI API key, since we will be using GPT-4 for our LLM-judged metrics.

In order to set your private key safely, please be sure to either export your key through a command-line terminal for your current instance, or, for a permanent addition to all user-based sessions, configure your favored environment management configuration file (i.e., .bashrc, .zshrc) to have the following entry:

`OPENAI_API_KEY=<your openai API key>`

In [1]:
%reload_ext autoreload
%autoreload 2

import getpass
import os
import sys
from pathlib import Path

import mlflow
import openai
import pandas as pd


In [2]:
os.environ["MLFLOW_TRACKING_URI"] = "http://0.0.0.0:5001"
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OPEN API key:")

Enter OPEN API key: ········


In [3]:
# Set up MLflow experiment
mlflow.set_experiment("6-question-answering-evaluation")

2025/06/19 13:40:27 INFO mlflow.tracking.fluent: Experiment with name '6-question-answering-evaluation' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/501136849596978774', creation_time=1750315227989, experiment_id='501136849596978774', last_update_time=1750315227989, lifecycle_stage='active', name='6-question-answering-evaluation', tags={}>

## Basic Question-Answering Evaluation

Create a test case of `inputs` that will be passed into the model and `ground_truth` which will be used to compare against the generated output from the model.

In [4]:
eval_df = pd.DataFrame(
    {
        "inputs": [
            "How does useEffect() work?",
            "What does the static keyword in a function mean?",
            "What does the 'finally' block in Python do?",
            "What is the difference between multiprocessing and multithreading?",
        ],
        "ground_truth": [
            "The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we’ll refer to it as our “effect”), and call it later after performing the DOM updates.",
            "Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.",
            "'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.",
            "Multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. Whereas multiprocessing refers to the ability of a system to run multiple processors in parallel, where each processor can run one or more threads.",
        ],
    }
)

Create a simple OpenAI model that asks gpt-4o to answer the question in two sentences. Call `mlflow.evaluate()` with the model and evaluation dataframe. 

In [5]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question in two sentences"
    basic_qa_model = mlflow.openai.log_model(
        model="gpt-4o-mini",
        task=openai.chat.completions,
        name="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",  # specify which column corresponds to the expected output
        model_type="question-answering",  # model type indicates which metrics are relevant for this task
        evaluators="default",
    )
results.metrics

/Users/artemsokolov/.pyenv/versions/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025/06/19 13:40:36 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-c780b421aedd46c28763f33ad8d67728
2025/06/19 13:40:36 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/19 13:40:36 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-c780b421aedd46c28763f33ad8d67728
2025/06/19 13:40:36 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/19 13:40:36 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call

🏃 View run vaunted-stork-664 at: http://0.0.0.0:5001/#/experiments/501136849596978774/runs/a83a39cfe8c64e57afd6924f54ec4cec
🧪 View experiment at: http://0.0.0.0:5001/#/experiments/501136849596978774


{'exact_match/v1': 0.0}

[Trace(trace_id=dde2a5653ded403ea91edad0f8a15d43), Trace(trace_id=6f04b88aeb884206bd8e2b3745be1c63), Trace(trace_id=bebf298e50a7411191fd1e71901f5935), Trace(trace_id=3063b28952ce44b28676c76a6b3d2dfc)]

Inspect the evaluation results table as a dataframe to see row-by-row metrics to further assess model performance

In [6]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React allows you to ...,70
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The `static` keyword in a function indicates t...,55
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used to defin...,61
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves running multiple proc...,76


## LLM-judged correctness with OpenAI GPT-4

Construct an answer similarity metric using the `answer_similarity()` metric factory function.

In [7]:
from mlflow.metrics.genai import EvaluationExample, answer_similarity

# Create an example to describe what answer_similarity means like for this problem.
example = EvaluationExample(
    input="What is MLflow?",
    output="MLflow is an open-source platform for managing machine "
    "learning workflows, including experiment tracking, model packaging, "
    "versioning, and deployment, simplifying the ML lifecycle.",
    score=4,
    justification="The definition effectively explains what MLflow is "
    "its purpose, and its developer. It could be more concise for a 5-score.",
    grading_context={
        "targets": "MLflow is an open-source platform for managing "
        "the end-to-end machine learning (ML) lifecycle. It was developed by Databricks, "
        "a company that specializes in big data and machine learning solutions. MLflow is "
        "designed to address the challenges that data scientists and machine learning "
        "engineers face when developing, training, and deploying machine learning models."
    },
)

# Construct the metric using OpenAI GPT-4 as the judge
answer_similarity_metric = answer_similarity(model="openai:/gpt-4", examples=[example])

print(answer_similarity_metric)

EvaluationMetric(name=answer_similarity, greater_is_better=True, long_name=answer_similarity, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's answer_similarity based on the rubric
justification: Your reasoning about the model's answer_similarity score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_similarity based on the input and output.
A definition of answer_similarity and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them be

Call `mlflow.evaluate()` again but with your new `answer_similarity_metric`

In [8]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[answer_similarity_metric],  # use the answer similarity metric created above
    )
results.metrics

2025/06/19 13:40:58 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-c780b421aedd46c28763f33ad8d67728
2025/06/19 13:40:58 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/19 13:40:58 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/06/19 13:41:01 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/06/19 13:41:01 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/06/19 13:41:01 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/06/19 13:41:01 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install te

🏃 View run smiling-shrimp-174 at: http://0.0.0.0:5001/#/experiments/501136849596978774/runs/7587b16f4172418bbb218fbd6b8b2c64
🧪 View experiment at: http://0.0.0.0:5001/#/experiments/501136849596978774


{'exact_match/v1': 0.0,
 'answer_similarity/v1/mean': np.float64(3.75),
 'answer_similarity/v1/variance': np.float64(1.1875),
 'answer_similarity/v1/p90': np.float64(4.7)}

[Trace(trace_id=cca0e8a5722749d0a34add9039fd94c7), Trace(trace_id=e6305636683a454aa7ccc1873b7c22a6), Trace(trace_id=a3ad061bcfad4636af552ad30046eb40), Trace(trace_id=546ee4639bcd4fbeb4f06013b38faa56)]

See the row-by-row LLM-judged answer similarity score and justifications

In [9]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,answer_similarity/v1/score,answer_similarity/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React is used to man...,63,4,The output effectively explains what the `useE...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword in a function indicates tha...,49,2,The output provides a correct explanation of t...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used to defin...,61,5,The output closely aligns with the provided ta...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves the concurrent execut...,66,4,The model's output accurately describes the co...


## Custom LLM-judged metric for professionalism

Create a custom metric that will be used to determine professionalism of the model outputs. Use `make_genai_metric` with a metric definition, grading prompt, grading example, and judge model configuration

In [10]:
from mlflow.metrics.genai import EvaluationExample, make_genai_metric

professionalism_metric = make_genai_metric(
    name="professionalism",
    definition=(
        "Professionalism refers to the use of a formal, respectful, and appropriate style of communication that is tailored to the context and audience. It often involves avoiding overly casual language, slang, or colloquialisms, and instead using clear, concise, and respectful language"
    ),
    grading_prompt=(
        "Professionalism: If the answer is written using a professional tone, below "
        "are the details for different scores: "
        "- Score 1: Language is extremely casual, informal, and may include slang or colloquialisms. Not suitable for professional contexts."
        "- Score 2: Language is casual but generally respectful and avoids strong informality or slang. Acceptable in some informal professional settings."
        "- Score 3: Language is balanced and avoids extreme informality or formality. Suitable for most professional contexts. "
        "- Score 4: Language is noticeably formal, respectful, and avoids casual elements. Appropriate for business or academic settings. "
        "- Score 5: Language is excessively formal, respectful, and avoids casual elements. Appropriate for the most formal settings such as textbooks. "
    ),
    examples=[
        EvaluationExample(
            input="What is MLflow?",
            output=(
                "MLflow is like your friendly neighborhood toolkit for managing your machine learning projects. It helps you track experiments, package your code and models, and collaborate with your team, making the whole ML workflow smoother. It's like your Swiss Army knife for machine learning!"
            ),
            score=2,
            justification=(
                "The response is written in a casual tone. It uses contractions, filler words such as 'like', and exclamation points, which make it sound less professional. "
            ),
        )
    ],
    version="v1",
    model="openai:/gpt-4",
    parameters={"temperature": 0.0},
    grading_context_columns=[],
    aggregations=["mean", "variance", "p90"],
    greater_is_better=True,
)

print(professionalism_metric)

EvaluationMetric(name=professionalism, greater_is_better=True, long_name=professionalism, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's professionalism based on the rubric
justification: Your reasoning about the model's professionalism score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called professionalism based on the input and output.
A definition of professionalism and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before complet

Call `mlflow.evaluate` with your new professionalism metric. 

In [11]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],  # use the professionalism metric we created above
    )
print(results.metrics)

2025/06/19 13:43:25 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-c780b421aedd46c28763f33ad8d67728
2025/06/19 13:43:25 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/19 13:43:25 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/06/19 13:43:28 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/06/19 13:43:28 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/06/19 13:43:28 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/06/19 13:43:28 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install te

🏃 View run bold-pig-5 at: http://0.0.0.0:5001/#/experiments/501136849596978774/runs/61c9b34a17e845439f3f773499718005
🧪 View experiment at: http://0.0.0.0:5001/#/experiments/501136849596978774
{'professionalism/v1/mean': np.float64(4.0), 'professionalism/v1/variance': np.float64(0.0), 'professionalism/v1/p90': np.float64(4.0)}


[Trace(trace_id=feb50b571d4246a2b3400e7ee85cae6e), Trace(trace_id=3cbc38f7ccbf4b78b6d40ae13f526fbc), Trace(trace_id=1449cd4e7be14ebaa9cafe8898d9a8f0), Trace(trace_id=89baede606f4403da8ce425959e65dff)]

In [12]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,professionalism/v1/score,professionalism/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React allows you to ...,60,4,The language used in the response is formal an...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The `static` keyword in a function indicates t...,64,4,The language used in the response is formal an...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used in excep...,62,4,The language used in the response is formal an...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves the use of multiple p...,63,4,The language used in the response is formal an...


Lets see if we can improve `basic_qa_model` by creating a new model that could perform better by changing the system prompt.

Call `mlflow.evaluate()` using the new model. Observe that the professionalism score has increased!

In [13]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question using extreme formality."
    professional_qa_model = mlflow.openai.log_model(
        model="gpt-4o-mini",
        task=openai.chat.completions,
        name="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        professional_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],
    )
print(results.metrics)

2025/06/19 13:43:45 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-04fe831f5c144294b895912474ed1bd4
2025/06/19 13:43:45 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/19 13:43:45 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-04fe831f5c144294b895912474ed1bd4
2025/06/19 13:43:45 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/19 13:43:45 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/06/19 13:43:52 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/06/19 13:43:52 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/06/19 13:43:52 WARNING mlflow.models.evaluation.utils.metric: Did not log metric '

🏃 View run thundering-snipe-413 at: http://0.0.0.0:5001/#/experiments/501136849596978774/runs/33e31cdfe6304c94a54b0ef0a68719ae
🧪 View experiment at: http://0.0.0.0:5001/#/experiments/501136849596978774
{'professionalism/v1/mean': np.float64(4.75), 'professionalism/v1/variance': np.float64(0.1875), 'professionalism/v1/p90': np.float64(5.0)}


[Trace(trace_id=bcc91468960d4c7c8fad2c9b4eb88e00), Trace(trace_id=32e102e77dd743aabdd25249ff30e4f3), Trace(trace_id=7d50eb4a5ef7400aa6185576b4ae7724), Trace(trace_id=52f176548bb34d20908bac442e611961)]

In [14]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,professionalism/v1/score,professionalism/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect` hook is an integral feature of...,320,5,The language used in the response is excessive...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...","The static keyword, when employed within the c...",167,4,The language used in the response is formal an...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python serves a quintes...,299,5,The language used in the response is excessive...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,The distinction between multiprocessing and mu...,234,5,The language used in the response is excessive...
